In [1]:
# imports
import tensorflow as tf
import keras
import pandas as pd
import json
import gzip

# word to idx
import pickle

# our functions
import functions as f


In [2]:
PATH = {}
PATH["dataset_classification"] = "dataset/classification/"
PATH["dataset_phase2"] = "dataset/phase2/"
PATH["dataset_labeling"] = "dataset/seq_labeling/"
PATH["music_reviews_train"] = PATH["dataset_classification"] + "music_reviews_train.json.gz"
PATH["music_reviews_dev"] = PATH["dataset_classification"] + "music_reviews_dev.json.gz"
PATH["music_reviews_test"] = PATH["dataset_classification"] + "music_reviews_test_masked.json.gz"
#PATH["news"] = PATH["dataset_phase2"] + "review_polarity.tar.gz"
cat_dict = {0: "negative", 1:"positive"}

In [3]:
# load our model
model = keras.models.load_model('model/2022-3-28-10-35-36')

# load our word to idx
sequencer = pickle.load( open( "model/text_to_seq.p", "rb" ) )
sequencer = pickle.loads(sequencer)

In [4]:
# Read in google drives
test = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQK7Cyu8BP41ZXqoS74SSHvoBRijNmIwU5p5PrB0RgnDbyWn0DdanuLxaaHTi1J5Fdw71IfsjoYhwJY/pub?gid=0&single=true&output=csv',
                   # Set first column as rownames in data frame
                   index_col=0, error_bad_lines=False
                  )
  # Same result as @TomAugspurger
raw = test.rename(columns={"Annotation (0: negative, 1: positive)": "annotation"})
raw.head(5)

c:\users\music\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,Sentence,annotation,category
index,,,
0,ok,1.0,misclassification
1,This is not sad,1.0,contrary of adjective
2,I am not happy to see her,0.0,contrary of adjective
3,This album is a riot to listen to,1.0,rare words
4,This CD will enchant anyone,1.0,rare words


1 load our model <br>
2 Load the word to idx saved object <br>
3 Find 100 sentencies, or write them. Also with tags ( positive-negative) <br>
4 Try to fuck it up ( swap the meaning, like "good" turn into "not good" and check the result <br>
5 Check accuracy, report hard cases 

In [23]:
'''# catch something from the DEV set
# dev = f.readJson(PATH["music_reviews_dev"])
data = dev
dev_sent = []
dev_sentiment = []
dev_idx = []
missing_indexies = []
dev_y_train = []
sent_dict = {"positive": 1, "negative": 0}
for i in range(len(data)):
    try:
        dev_sent.append(data[i]["reviewText"])
        dev_sentiment.append(data[i]["sentiment"])
        dev_idx.append(i)
        dev_y_train.append(sent_dict[data[i]["sentiment"]]) 
    except KeyError:
        missing_indexies.append(i)
        continue
print(missing_indexies)

test_x = dev_sent
test_y = dev_sentiment'''

'# catch something from the DEV set\n# dev = f.readJson(PATH["music_reviews_dev"])\ndata = dev\ndev_sent = []\ndev_sentiment = []\ndev_idx = []\nmissing_indexies = []\ndev_y_train = []\nsent_dict = {"positive": 1, "negative": 0}\nfor i in range(len(data)):\n    try:\n        dev_sent.append(data[i]["reviewText"])\n        dev_sentiment.append(data[i]["sentiment"])\n        dev_idx.append(i)\n        dev_y_train.append(sent_dict[data[i]["sentiment"]]) \n    except KeyError:\n        missing_indexies.append(i)\n        continue\nprint(missing_indexies)\n\ntest_x = dev_sent\ntest_y = dev_sentiment'

In [5]:
test_x = []
test_y = []
test_com = []
for i in range(len(raw)):
    try: 
        len(raw['Sentence'][i])
        test_x.append(raw['Sentence'][i])
        test_y.append(raw['annotation'][i])
        test_com.append(raw['category'][i])
    except:
        continue

In [24]:
test_x[32:33]

['Its so funky, but it is not written on the cd. I am a metal guy, but still good']

Examples from checklist repo

In [39]:
# Sentencies
Train = ['This is a good day', 'what a hell is going on with the sound of the guitar? I never ever heard this shit before, unbeliveable!']
#Train = ['This is bullshit, it sound like an old creppy dog']
Labels = ['positive','negative']
#Labels = ['negative']
Train = ['I was not so nervous when I hear her to shout with me. After we had conversation about the problem, and we reconciled. Now we can speak each other']
#Train = ['I was not so nervous when I hear her to shout with me. After we had conversation about the problem. Now we can speak each other']
#Train = ['I was not so nervous when I hear her to shout with me. After we had conversation about the problem, now everything is more or lessfine']
#Train = ['The atmosphere is attractive, but a little uncomfortable. Anyway, the trip was cheap, we are together, so it is more or less good']
#Train = ['I was not so nervous when I hear her to shout with me, its crazy']
Train = ["Is it a perfect music? This album don't give the answer, don't listen, don't buy and don't waste your time with it"]
#Train = ['This is  A good song not really my type of music but I listen to it some times, I give it 2to4 stars']
#Train = test_x[38]
#Train = ["My dentist recommended this as a relaxation technique for dental visits. They give me an ipod with headphones, play this on it and it relieves some of the stress of dental treatment, which I dislike intensely. It worked so well that I bought my own copy to try at home. I fall asleep after a couple of minutes and stay asleep. Instead of tossing and turning, I hardly move at all. Highly recommend."]
Labels = ['positive']
Labels_i = [1]
# labels to idx
labels = []
for i in Labels:
    if i == "positive":
        labels.append(1)
    else:
        labels.append(0)


# Transform the example sentencies for idx
t = ["This is an easy day? Yeah, it is easier than a day in a middle of a war without food and weapon"]
t = ["Is this an easy day? Yeah, it is easier than a day in a middle of a war without food and weapon"]
t = test_x[36:37]
t = ["Is it a perfect music? This album don't give the answer, don't listen, don't buy and don't waste your time with it"]

Train_s = sequencer.transform(t)
print
#Train_s = sequencer.transform(test_x)
# padding apply
def sequence_pad(sequence, padding='pre', maxlen=50):
    res = tf.keras.preprocessing.sequence.pad_sequences(
        sequence,
        maxlen=maxlen,
        dtype='int32',
        padding=padding,
        truncating='pre',
        value=0.0)
    return res
padding = "post"
maxlen = 50

X_train_p = sequence_pad(Train_s, padding=padding, maxlen=maxlen)

# prediction
result = model.predict(X_train_p)
print(result)
print(t)
#print(len(result))

[[0.8800013]]
["Is it a perfect music? This album don't give the answer, don't listen, don't buy and don't waste your time with it"]


In [36]:
for i in range(len(t)):
    if 0.40 <= result[i] <= 0.60:
        print(test_x[i])
        print(result[i])

I used to love everything composed by this artist, but these latest songs have just been bad overall
[0.51934355]
I was not so nervous when I hear her to shout with me. After we had conversation about the problem, now everything is more or lessfine
[0.45422605]
The atmosphere is attractive, but a little uncomfortable. Anyway, the trip was cheap, we are together, so it is good
[0.56961346]
This is  A good song not really my type of music but I listen to it some times, I give it 2to4 stars
[0.5097012]
This album is sick nasty! I've been a fan before they even got signed! cannot wait to hear more from these guys, I don't regret buying this album at all, I love every single second of it :]
[0.48066774]
Bought for a friend but I didn't care for it. He loved it.
[0.4915705]
Cool old song. HOWEVER, I think this is a remixed or re-recorded version. It doesn't seem to have the same punch as the radio versus I hear every once in a while.
[0.52943873]
Not bad, but I thought I was downloading the 

In [51]:
'''k= 9237
print(result[k])

print(dev_sent[k])
print(dev_sentiment[k])'''

'k= 9237\nprint(result[k])\n\nprint(dev_sent[k])\nprint(dev_sentiment[k])'

In [52]:
'''# DEV SET ana
c = 0
index_of_original = []
for i in range(len(dev_sent)):
#for i in range(1):
    if i in missing_indexies:
        print("-------------------------what?")
        c += 1
        continue
    else:
        if 0.47 <= result[i] <= 0.49:
            print("-----------index,", i)
            print("-----------indexoriginal,", c+i)
            print("--- original sentiment:",test_y[i])
            print(dev_sent[i])
            #print(Train[i])
            print(result[i])
            print(len(dev_sent[i].split()))
            index_of_original.append(i+c)'''

'# DEV SET ana\nc = 0\nindex_of_original = []\nfor i in range(len(dev_sent)):\n#for i in range(1):\n    if i in missing_indexies:\n        print("-------------------------what?")\n        c += 1\n        continue\n    else:\n        if 0.47 <= result[i] <= 0.49:\n            print("-----------index,", i)\n            print("-----------indexoriginal,", c+i)\n            print("--- original sentiment:",test_y[i])\n            print(dev_sent[i])\n            #print(Train[i])\n            print(result[i])\n            print(len(dev_sent[i].split()))\n            index_of_original.append(i+c)'

## Correct or wrong ? -- CSV

In [53]:
threshold = 0.5
acc = 0
acc_count = 0
res_final = []
wrongly_classified_text = []
wrongly_classified_original_sentiment = []
all_text = []
all_sentiment = []
all_categories = []
for r in result:
    acc_count+=1
    if r<threshold:
        r = 0
        #res_final.append("negative")
        res_final.append(0)
    if r>threshold:
        #res_final.append("positive")
        res_final.append(1)
        
fault_counter = 0
cases = 0
for i in range(len(result)):
    #print("index", i)
    #print("####", res_final[i], "---", hard_sen_sent[i])
    if test_y[i] == res_final[i]:
        #print(" test", test_y[i], "----- res", res_final[i])
        #print("correct")
        
        ###### ALL CASES
        all_text.append(test_x[i])
        all_sentiment.append(cat_dict[test_y[i]]) # need to be updated for dict
        all_categories.append(test_com[i])
        cases +=1
        ######
        continue
    else:
        ##################  for the csv set
        print("--- wrong, sentence: \n ",test_x[i], "\n---- original sentiment ", test_y[i], "\n result raw: ", result[i])
        # for testing manually 
        #print("--- wrong, sentence: \n ",Train[i], "\n---- original sentiment ", Labels[i], "\n result raw: ", result[i])
        fault_counter += 1
        wrongly_classified_text.append(test_x[i])
        wrongly_classified_original_sentiment.append(test_y[i])
        ###### ALL CASES
        all_text.append(test_x[i])
        all_sentiment.append(cat_dict[test_y[i]]) # need to be updated for dict
        all_categories.append(test_com[i])
        cases +=1
        ######
print(fault_counter)

--- wrong, sentence: 
  ok 
---- original sentiment  1.0 
 result raw:  [0.1330792]
--- wrong, sentence: 
  This is not sad 
---- original sentiment  1.0 
 result raw:  [0.03364298]
--- wrong, sentence: 
  I used to love everything composed by this artist, but these latest songs have just been bad overall 
---- original sentiment  0.0 
 result raw:  [0.51934355]
--- wrong, sentence: 
  I hated this album when I bought it, but it's actually managed to win me over after listening to it a couple of times 
---- original sentiment  1.0 
 result raw:  [0.1203514]
--- wrong, sentence: 
  This album is so ominous that i've been too afraid to go shopping or even turn off the lights for the past 3 days. You absolutely have to listen to this if you are a fan of horror 
---- original sentiment  1.0 
 result raw:  [0.10521677]
--- wrong, sentence: 
  I simply can't understand this album. I hate the choice of instruments, the song genres are all over the place, yet the songs are still repetitve, but

# Write to JSON, save and check it

In [54]:
keys = ["reviewText","sentiment","category"]

In [59]:
json_raw_data = []
for i in range(cases):
    temp_data = []
    def_dict = dict()
    for c in keys:
        if c ==  "reviewText":
            def_dict[c] = str(all_text[i])
        if c ==  "sentiment":
            def_dict[c] = str(all_sentiment[i])
        if c ==  "category":
            def_dict[c] = str(all_categories[i])   
    json_raw_data.append(def_dict) # ok this is very good, itsa opening with json.load
    
outFile = open('group16.json', 'w')
for instance in json_raw_data:
      outFile.write(json.dumps(instance) + '\n')
outFile.close()

## Check format

In [60]:
import json
inputPath = 'group16.json'

for lineIdx, line in enumerate(open(inputPath)):
    try:
        data = json.loads(line)
        #print(data)
    except ValueError as e:
        print('error, instance ' + str(lineIdx+1) + ' is not in valid json format')
        continue
    if 'reviewText' not in data:
        print("error, instance " + str(lineIdx+1) + ' does not contain key "reviewText"')
        continue
    if 'sentiment' not in data:
        print("error, instance " + str(lineIdx+1) + ' does not contain key "sentiment"')
        continue
    if data['sentiment'] not in ['positive', 'negative']:
        print("error, instance " + str(lineIdx+1) + ': sentiment is not positive/negative')
        continue
        
if lineIdx+1 < 100:
    print('Too little instances(' + str(lineIdx) + '), please generate more')
if lineIdx+1 > 1000:
    print('Too many instances(' + str(lineIdx) + '), please generate more')